In [1]:
import pygame;
from pygame.locals import *
from pgu import engine
import game_board
import play_vsCPU
import play_pvp

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Tic_Tac_Toe(engine.Game):
    def __init__(self):
        super().__init__()
        pygame.mixer.pre_init(44100, -16, 2, 512)
        pygame.init()
        self.myTitleFont = pygame.font.Font('im&font/Raind___.TTF',80)
        self.myTextFont = pygame.font.Font('im&font/Raind___.TTF',40)
        self.screen = pygame.display.set_mode((750,750), SWSURFACE)
        self.crono = pygame.time.Clock()
        self._init_state_machine()
        self.result = ''
        
    def _init_state_machine(self):
        self.menu_state = Menu(self)
        self.playervsCPU_state = play_vsCPU.PlayerVSCPU(self)
        self.playervsplayer_state = play_pvp.PlayerVSPlayer(self)
        self.result_state = Result(self)
        self.quit_state = engine.Quit(self)
    
    def change_state(self, transition=None):
        """
        Implements the state machine of the game.
        Given self.state and an optional parameter indicating 
        the kind of transition, computes and returns the new state
        """
        if self.state is self.menu_state:
            if transition == 'EXIT':
                new_state = self.quit_state
            elif transition == 'PLAYERVSCPU':
                new_state = self.playervsCPU_state
                new_state.init()
            elif transition == 'PLAYERVSPLAYER':
                new_state = self.playervsplayer_state
                new_state.init()
            else:
                raise ValueError('Unknown transition indicator')
        elif self.state is self.playervsCPU_state:
            if transition == 'EXIT':
                new_state = self.quit_state
            elif transition == 'RESULT':
                new_state = self.result_state
            else:
                raise ValueError('Unknown transition indicator')
        elif self.state is self.playervsplayer_state:
            if transition == 'EXIT':
                new_state = self.quit_state
            elif transition == 'RESULT':
                new_state = self.result_state
            else:
                raise ValueError('Unknown transition indicator')
        elif self.state is self.result_state:
            if transition == 'EXIT':
                new_state = self.quit_state
            elif transition == 'RETRY':
                if self.result == 'Player wins' or self.result == 'CPU wins' or self.result == 'DrawPvsCPU':
                    new_state = self.playervsCPU_state
                    new_state.init()
                else:
                    new_state = self.playervsplayer_state
                    new_state.init()
            elif transition == 'MENU':
                new_state = self.menu_state
            else:
                raise ValueError('Unknown transition indicator')
        else:
            raise ValueError('Unknown game state value')
        return new_state
        
    def run(self):
        # Calls the main loop with the initial state
        # (self.menu, in this case)
        super().run(self.menu_state, self.screen)

##------------------------------------------------------Menu screen-------------------------------------------

class Menu(engine.State):
    def init(self):
        self.menuTitle = self.game.myTitleFont.render('Tic-Tac-Toe',0,(255,255,0))
        self.menuText1 = self.game.myTextFont.render('P vs CPU',0,(255,255,255))
        self.menuText2 = self.game.myTextFont.render('P vs P',0,(255,255,255))
        self.pvCPU_button = game_board.Button("im&font/button.png",(250,500))
        self.pvp_button = game_board.Button("im&font/button.png",(500,500))
        self.buttons = pygame.sprite.Group()
        self.buttons.add(self.pvCPU_button)
        self.buttons.add(self.pvp_button)
        
    def paint(self, s):
        s.fill((0, 0, 0))
        s.blit(self.menuTitle,(70,170))

    def event(self,e): 
        if e.type is KEYDOWN:
            if e.key == K_ESCAPE:
                return self.game.change_state('EXIT')
        elif e.type is pygame.MOUSEBUTTONDOWN:
            for b in list(self.buttons):
                    if b.rect.collidepoint(e.pos):
                        if  b == self.pvCPU_button:
                            return self.game.change_state('PLAYERVSCPU')
                        else:
                            return self.game.change_state('PLAYERVSPLAYER')
    def loop(self):
        pass
    
    def update(self,screen):
        self.paint(screen)
        self.buttons.draw(screen)
        screen.blit(self.menuText1,(148,478))
        screen.blit(self.menuText2,(428,478))
        
        pygame.display.flip()

##------------------------------------------------------Result screen ---------------------------------

class Result(engine.State):
    def init(self):
        self.resultText1 = self.game.myTextFont.render('Retry',0,(255,0,0))
        self.resultText2 = self.game.myTextFont.render('Menu',0,(255,255,255))
        self.retry_button = game_board.Button("im&font/button.png",(250,500))
        self.menu_button = game_board.Button("im&font/button.png",(500,500))
        self.buttons = pygame.sprite.Group()
        self.buttons.add(self.retry_button)
        self.buttons.add(self.menu_button)
        
    def paint(self, s):
        s.fill((0, 0, 0))

    def event(self,e): 
        if e.type is KEYDOWN:
            if e.key == K_ESCAPE:
                return self.game.change_state('EXIT')
        elif e.type is pygame.MOUSEBUTTONDOWN:
            for b in list(self.buttons):
                    if b.rect.collidepoint(e.pos):
                        if  b == self.retry_button:
                            return self.game.change_state('RETRY')
                        elif b == self.menu_button:
                            return self.game.change_state('MENU')
    def loop(self):
        pass
    
    def update(self,screen):
        self.paint(screen)
        self.buttons.draw(screen)
        screen.blit(self.resultText1,(182,478))
        screen.blit(self.resultText2,(442,478))
        if self.game.result == 'DrawPvP' or self.game.result == 'DrawPvsCPU':
            result = self.game.myTitleFont.render('Draw!',0,(255,255,0))
            screen.blit(result,(250,170))
        else:
            result = self.game.myTitleFont.render(self.game.result+'!',0,(255,255,0))
            screen.blit(result,(30,170))
        pygame.display.flip()
        
def main():
    game = Tic_Tac_Toe()
    game.run()


if __name__ == "__main__":
    main()